In [5]:
# --- Бібліотеки для даної роботи ---
try:
    import numpy, pandas, scipy, matplotlib, jinja2
    print("Бібліотеки вже встановлені. Пропускаємо інсталяцію.")
except ImportError:
    print("Встановлюємо бібліотеки...")
    %pip install numpy pandas scipy matplotlib jinja2

Бібліотеки вже встановлені. Пропускаємо інсталяцію.


## Задача 2. Моделювання процесу навчання

**Умова:**

Ви розробляє адаптивну систему навчання для EdTech-платформи. Потрібно змоделювати, як студент засвоює новий матеріал з часом. Педагогічні дослідження показують, що швидкість навчання пропорційна кількості ще **не** засвоєного матеріалу.

Цю залежність описує наступне диференціальне рівняння:
$$\frac{dK}{dt} = r(M - K)$$
де:
* $K(t)$ — рівень знань студента в момент часу $t$ (у відсотках, від 0 до 100)
* $M = 100$ — максимальний рівень знань (100%)
* $r = 0.15$ — коефіцієнт швидкості навчання (1/день)
* Початкова умова: $K(0) = 10$ (студент починає з 10% базових знань)

Наше рівняння каже нам, що чим більше студент вже знає, тим повільніше він вчить нове.

**Використовуючи чисельні методи розв'язання диференціальних рівнянь, напишіть програму та виконайте наступні кроки:**

1.  **Реалізуйте модель.** Створіть функцію `learning_rate(t, K)`, яка обчислює праву частину диференціального рівняння $\frac{dK}{dt} = r(M - K)$ для заданих параметрів $M = 100$ та $r = 0.15$. *Зверніть увагу*, що функція `solve_ivp` вимагає, щоб права частина рівняння мала сигнатуру `f(t, y)` — два параметри: час та значення функції. У нашому рівнянні швидкість навчання не залежить від часу явно (таке рівняння називають *автономним*), але параметр `t` все одно має бути присутній у сигнатурі функції.
2.  **Розв'яжіть рівняння чисельно.** Використовуючи функцію `solve_ivp` з бібліотеки `scipy.integrate`, знайдіть розв'язок $K(t)$ на проміжку від 0 до 30 днів з початковою умовою $K(0) = 10$.
3.  **Дослідіть вплив початкового рівня.** Порівняйте час досягнення 90% знань для трьох студентів з різним початковим рівнем: $K(0) = 5, K(0) = 10, K(0) = 20$. Зробіть висновок про вплив початкової підготовки.

In [6]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from IPython.display import display

M = 100
r = 0.15
target_knowledge = 90.0

def learning_rate(t, K):
    return r * (M - K)

def reach_90_percent(t, K):
    return K[0] - target_knowledge

reach_90_percent.terminal = True
reach_90_percent.direction = 1

initial_conditions = [5, 10, 20]
t_span = (0, 60)
results = []
raw_solutions = {}

for k0 in initial_conditions:
    sol = solve_ivp(
        fun=learning_rate,
        t_span=t_span,
        y0=[k0],
        events=reach_90_percent,
        dense_output=True,
        rtol=1e-6, 
        atol=1e-6
    )
    
    time_to_90 = sol.t_events[0][0] if sol.t_events[0].size > 0 else None
    
    results.append({
        "Початковий рівень K(0)": f"{k0}%",
        "Макс. знання (M)": f"{M}%",
        "Час досягнення 90% (днів)": time_to_90,
        "Швидкість (r)": r
    })
    
    raw_solutions[f"K0_{k0}"] = {
        "успіх": sol.success,
        "повідомлення": sol.message,
        "час_подій": sol.t_events,
        "фінальне_значення": sol.y[:,-1][0]
    }

df_results = pd.DataFrame(results)

print("Красивий Вивід:")
print("-" * 60)
print("АНАЛІЗ ЕФЕКТИВНОСТІ НАВЧАННЯ (EDTECH MODEL)")
print("-" * 60)

styled_df = df_results.style.format({
    "Час досягнення 90% (днів)": "{:.2f}"
}).background_gradient(subset=["Час досягнення 90% (днів)"], cmap="RdYlGn_r")

display(styled_df)

fastest_student = df_results.loc[df_results['Час досягнення 90% (днів)'].idxmin()]
slowest_student = df_results.loc[df_results['Час досягнення 90% (днів)'].idxmax()]
diff = slowest_student['Час досягнення 90% (днів)'] - fastest_student['Час досягнення 90% (днів)']

print(f"\nВИСНОВОК:")
print(f"Студент з початковим рівнем {fastest_student['Початковий рівень K(0)']} досягає мети найшвидше.")
print(f"Різниця між стартом з 5% і 20% складає {diff:.2f} днів.")
print("Чим вищий початковий рівень, тим менше часу потрібно для опанування матеріалу,")
print("оскільки зменшується обсяг незнайомої інформації (M - K).")
print("=" * 60)

print("\nТехнічний вивід:")
print(raw_solutions)

Красивий Вивід:
------------------------------------------------------------
АНАЛІЗ ЕФЕКТИВНОСТІ НАВЧАННЯ (EDTECH MODEL)
------------------------------------------------------------


,Початковий рівень K(0),Макс. знання (M),Час досягнення 90% (днів),Швидкість (r)
0,5%,100%,15.01,0.150000
1,10%,100%,14.65,0.150000
2,20%,100%,13.86,0.150000



ВИСНОВОК:
Студент з початковим рівнем 20% досягає мети найшвидше.
Різниця між стартом з 5% і 20% складає 1.15 днів.
Чим вищий початковий рівень, тим менше часу потрібно для опанування матеріалу,
оскільки зменшується обсяг незнайомої інформації (M - K).

Технічний вивід:
{'K0_5': {'успіх': True, 'повідомлення': 'A termination event occurred.', 'час_подій': [array([15.00860643])], 'фінальне_значення': np.float64(90.0)}, 'K0_10': {'успіх': True, 'повідомлення': 'A termination event occurred.', 'час_подій': [array([14.64817216])], 'фінальне_значення': np.float64(90.0)}, 'K0_20': {'успіх': True, 'повідомлення': 'A termination event occurred.', 'час_подій': [array([13.86294612])], 'фінальне_значення': np.float64(90.0)}}


### Аналітичне обґрунтування

Маємо лінійне неоднорідне диференціальне рівняння першого порядку зі сталими коефіцієнтами (або рівняння з відокремлюваними змінними):

$$\frac{dK}{dt} = r(M - K)$$

**1. Розв'язок методом відокремлення змінних:**

Перенесемо змінні $K$ в одну сторону, а $t$ в іншу:
$$\frac{dK}{M - K} = r \, dt$$

Проінтегруємо обидві частини:
$$\int \frac{dK}{M - K} = \int r \, dt$$

$$-\ln|M - K| = rt + C$$

Позбудемося логарифма:
$$M - K = e^{-(rt + C)} = e^{-C} \cdot e^{-rt}$$

Позначимо $e^{-C}$ як довільну сталу $A$. Враховуючи, що $K < M$, модуль можна опустити:
$$K(t) = M - A \cdot e^{-rt}$$

**2. Знаходження сталої з початкової умови:**

При $t = 0, K(0) = K_0$. Підставимо у рівняння:
$$K_0 = M - A \cdot e^{0} \Rightarrow K_0 = M - A$$
$$A = M - K_0$$

**3. Аналітична формула моделі:**

$$K(t) = M - (M - K_0)e^{-rt}$$

**4. Розрахунок часу для досягнення 90% ($K=90$):**

Виразимо $t$ з формули:
$$90 = 100 - (100 - K_0)e^{-0.15t}$$
$$(100 - K_0)e^{-0.15t} = 10$$
$$e^{-0.15t} = \frac{10}{100 - K_0}$$
$$-0.15t = \ln\left(\frac{10}{100 - K_0}\right)$$
$$t = \frac{\ln(100 - K_0) - \ln(10)}{0.15}$$

**Перевірка для $K_0 = 10$:**

$$t = \frac{\ln(90) - \ln(10)}{0.15} = \frac{\ln(9)}{0.15} \approx \frac{2.197}{0.15} \approx 14.65 \text{ днів}$$

Це значення має збігатися з чисельним результатом, отриманим у коді.